### import statements

In [1]:
%matplotlib inline
import sys
import os
import pandas as pd
sys.path.append('../src')
import datetime
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pickle

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
from plot_utils import *

### constants

In [3]:
F_INPUT = '../../data/v2/data.h5'
ds_label_all = 'r0' # ds stands for data source
ds_label_day = 'r0_day'
ds_label_ngt = 'r0_night'

In [4]:
DENSE_SUBSET_START = '2018-05-03'
DENSE_SUBSET_END = '2020-12-30'

In [5]:
smooth_mins = [1, 2, 5, 10, 15, 20]
r0_map = {'all':'r0_all','day':'r0_day','ngt':'r0_ngt'} # adn = all, day, ngt

In [6]:
feats_mnus_cn2 = ['pressure', 'relative_humidity', 'temperature', 'wind_speed', 'solar_zenith_angle','dayofyear', 'hour']
feats_plus_cn2 = feats_mnus_cn2 + ['logCn2']
feats_map = {'feats_plus_cn2': feats_plus_cn2, 'feats_mnus_cn2': feats_mnus_cn2}

In [7]:
# xmin, ymin = 0, 0
# xmax, ymax = 20, 20

### read data

In [8]:
df = pd.read_hdf(F_INPUT, 'merged')

### add new features

In [9]:
df['dayofyear'] = df.index.dayofyear
df['hour'] = df.index.hour
df['logCn2'] = np.log10(df['Cn2'])

### applying smoothing

In [10]:
for m in smooth_mins:
    for ds_label in [ds_label_all, ds_label_day, ds_label_ngt]:
        if ds_label == 'r0':
            label = ds_label + '_all_{}T'.format(m)
        elif ds_label == 'r0_night':
            label = 'r0_ngt_{}T'.format(m)
        else:
            label = ds_label + '_{}T'.format(m)
        if m == 1:
            df[label] = df[ds_label]
        else:
            df[label] = df[ds_label].rolling('{}T'.format(m)).mean()

In [11]:
df.columns

Index(['o(I)_I', 'r0', 'solar_zenith_angle', 'r0_day', 'r0_night', 'pressure',
       'relative_humidity', 'temperature', 'wind_direction', 'wind_speed',
       'Cn2', 'dayofyear', 'hour', 'logCn2', 'r0_all_1T', 'r0_day_1T',
       'r0_ngt_1T', 'r0_all_2T', 'r0_day_2T', 'r0_ngt_2T', 'r0_all_5T',
       'r0_day_5T', 'r0_ngt_5T', 'r0_all_10T', 'r0_day_10T', 'r0_ngt_10T',
       'r0_all_15T', 'r0_day_15T', 'r0_ngt_15T', 'r0_all_20T', 'r0_day_20T',
       'r0_ngt_20T'],
      dtype='object')

In [12]:
df.describe()

,o(I)_I,r0,solar_zenith_angle,r0_day,r0_night,pressure,relative_humidity,temperature,wind_direction,wind_speed,...,r0_ngt_5T,r0_all_10T,r0_day_10T,r0_ngt_10T,r0_all_15T,r0_day_15T,r0_ngt_15T,r0_all_20T,r0_day_20T,r0_ngt_20T
count,342443.000000,483758.000000,483758.000000,342443.000000,141315.000000,1.740394e+06,1.653058e+06,1.653063e+06,1.783564e+06,1.783640e+06,...,258373.000000,688752.000000,417620.000000,271132.000000,715555.000000,436672.000000,278883.000000,736339.000000,451326.000000,285013.000000
mean,0.001379,4.729702,71.757210,3.651714,7.341948,7.746396e+02,3.542895e+01,1.103738e+01,1.908795e+02,2.313792e+00,...,7.335459,5.099589,3.656218,7.322788,5.086838,3.664591,7.313778,5.081379,3.675858,7.307061
std,0.000450,2.933261,41.077000,2.356312,2.521823,4.565213e+00,2.407449e+01,7.998990e+00,8.232246e+01,1.720400e+00,...,2.474673,3.120671,2.624671,2.444669,3.124712,2.654920,2.426685,3.122820,2.670015,2.413242
min,0.000102,0.821452,10.943276,0.821452,2.367400,7.516000e+02,1.000000e+00,-1.300000e+01,1.450000e+00,3.500000e-02,...,2.367400,0.821452,0.821452,2.367400,0.821452,0.821452,2.367400,0.821452,0.821452,2.367400
25%,0.001060,2.571765,42.658571,2.299625,5.500200,7.721000e+02,1.900000e+01,5.400000e+00,1.401579e+02,1.185000e+00,...,5.532833,2.712008,2.322928,5.543217,2.702915,2.322861,5.551622,2.700245,2.324379,5.560275
50%,0.001353,3.801627,57.746335,3.012749,7.057700,7.753000e+02,2.900000e+01,1.110000e+01,1.879474e+02,1.845000e+00,...,7.072533,4.245727,2.968483,7.074229,4.221276,2.966060,7.076489,4.212579,2.968290,7.076489
75%,0.001649,6.273073,112.630556,4.262205,8.908900,7.780000e+02,4.500000e+01,1.740000e+01,2.556842e+02,2.865000e+00,...,8.888133,6.924200,4.122536,8.871520,6.915300,4.123222,8.856551,6.913749,4.133728,8.851050
max,0.003500,86.039498,169.053912,86.039498,24.857700,7.856000e+02,1.100000e+02,3.180000e+01,3.600000e+02,3.196667e+01,...,24.857700,86.039498,86.039498,22.076450,86.039498,86.039498,22.076450,86.039498,86.039498,20.808942


### restricting data to usable, relatively dense subset

In [13]:
df_subset = df[(df.index > DENSE_SUBSET_START) & (df.index < DENSE_SUBSET_END)]
df_subset.describe()

,o(I)_I,r0,solar_zenith_angle,r0_day,r0_night,pressure,relative_humidity,temperature,wind_direction,wind_speed,...,r0_ngt_5T,r0_all_10T,r0_day_10T,r0_ngt_10T,r0_all_15T,r0_day_15T,r0_ngt_15T,r0_all_20T,r0_day_20T,r0_ngt_20T
count,273728.000000,413091.000000,413091.000000,273728.000000,139363.000000,1.252046e+06,1.164787e+06,1.164792e+06,1.280385e+06,1.280461e+06,...,254844.000000,598586.000000,331070.000000,267516.000000,620687.000000,345468.000000,275219.000000,637842.000000,356538.000000,281304.000000
mean,0.001382,4.871890,74.698190,3.607425,7.355473,7.744021e+02,3.532743e+01,1.104439e+01,1.747172e+02,2.327763e+00,...,7.350375,5.288679,3.633298,7.337330,5.277392,3.643771,7.327991,5.272362,3.655958,7.321068
std,0.000448,2.995811,43.192050,2.357307,2.525811,4.687234e+00,2.406511e+01,7.993629e+00,6.763609e+01,1.770945e+00,...,2.478042,3.153837,2.647643,2.448015,3.154897,2.675779,2.430055,3.152587,2.692277,2.416628
min,0.000102,0.821452,10.943276,0.821452,2.367400,7.516000e+02,1.000000e+00,-1.300000e+01,2.050000e+00,3.500000e-02,...,2.367400,0.821452,0.821452,2.367400,0.821452,0.821452,2.367400,0.821452,0.821452,2.367400
25%,0.001066,2.599046,42.089154,2.273231,5.509100,7.718000e+02,1.900000e+01,5.200000e+00,1.163684e+02,1.163158e+00,...,5.544700,2.774620,2.302938,5.557414,2.766120,2.303350,5.564280,2.763078,2.305506,5.571400
50%,0.001360,3.969400,58.682654,2.966270,7.066600,7.751000e+02,2.900000e+01,1.110000e+01,1.835500e+02,1.826316e+00,...,7.084400,4.590917,2.927821,7.087367,4.576505,2.924869,7.089344,4.567925,2.926963,7.088850
75%,0.001653,6.553345,118.005308,4.196203,8.926700,7.779000e+02,4.500000e+01,1.750000e+01,2.090526e+02,2.880000e+00,...,8.905933,7.212560,4.079838,8.889617,7.207022,4.084673,8.876267,7.209000,4.099386,8.869192
max,0.003500,86.039498,169.053912,86.039498,24.857700,7.838000e+02,1.100000e+02,3.180000e+01,3.600000e+02,3.196667e+01,...,24.857700,86.039498,86.039498,22.076450,86.039498,86.039498,22.076450,86.039498,86.039498,20.808942


### resampling back down to 5 mins

In [14]:
df_subset_resamp5 = df_subset.resample('5 min').median()
df_subset_resamp5.describe()

,o(I)_I,r0,solar_zenith_angle,r0_day,r0_night,pressure,relative_humidity,temperature,wind_direction,wind_speed,...,r0_ngt_5T,r0_all_10T,r0_day_10T,r0_ngt_10T,r0_all_15T,r0_day_15T,r0_ngt_15T,r0_all_20T,r0_day_20T,r0_ngt_20T
count,61816.000000,112767.000000,112767.000000,61816.000000,50951.000000,256360.000000,239076.000000,239077.000000,256372.000000,256386.000000,...,53106.000000,123307.000000,68556.000000,54751.000000,126881.000000,70871.000000,56010.000000,129839.000000,72747.000000,57092.000000
mean,0.001417,5.282660,84.396171,3.577739,7.351144,774.401450,35.501078,11.022659,174.671009,2.320158,...,7.339237,5.274953,3.634919,7.328508,5.270614,3.649666,7.321646,5.270526,3.665283,7.315937
std,0.000480,3.156839,44.883292,2.572347,2.494635,4.685693,24.264907,8.002353,65.553669,1.746939,...,2.480251,3.183171,2.713971,2.451988,3.181179,2.735333,2.434412,3.175887,2.746550,2.420576
min,0.000103,0.821452,10.945537,0.821452,2.367400,752.100000,1.000000,-13.000000,6.000000,0.100000,...,2.367400,0.821452,0.821452,2.367400,0.821452,0.821452,2.367400,0.821452,0.821452,2.367400
25%,0.001100,2.742663,46.747923,2.249879,5.535800,771.800000,19.000000,5.200000,117.100000,1.170000,...,5.540250,2.756131,2.286729,5.547667,2.754904,2.291331,5.558050,2.755582,2.297297,5.564725
50%,0.001391,4.598131,67.380905,2.890970,7.084400,775.100000,29.000000,11.100000,183.700000,1.820000,...,7.081433,4.565700,2.916605,7.082917,4.561744,2.918465,7.084845,4.563890,2.922754,7.085142
75%,0.001665,7.213450,124.278989,4.056879,8.913350,777.800000,45.000000,17.500000,207.526316,2.866667,...,8.900000,7.204868,4.077276,8.885463,7.206033,4.088772,8.873300,7.207808,4.106606,8.865285
max,0.003500,81.041189,169.048530,81.041189,23.229000,783.800000,110.000000,31.500000,353.789474,20.285000,...,22.076450,77.232305,77.232305,22.076450,77.232305,77.232305,22.076450,77.232305,77.232305,20.613085


### taking the train and test

In [15]:
split_date = '2019-12-31'
train = df_subset.index <= split_date
test  = df_subset.index > split_date

### creating non-nan masks

In [16]:
valid_masks = {}
for f in feats_map.keys(): 
    valid_masks[f] = {}
    for r in r0_map.keys():
        valid_masks[f][r] = {}
        for m in smooth_mins:
            label = '{}_{}T'.format(r0_map[r], str(m))
            feats_plus_label = feats_map[f] + [label]
            valid_masks[f][r][m] = ~df_subset[feats_plus_label].isnull().any(axis=1)
            print("{} {} {:2} {:7} {:7}".format(f, r, m, np.sum(train & valid_masks[f][r][m]), np.sum(test & valid_masks[f][r][m])))

feats_plus_cn2 all  1  251681   98859
feats_plus_cn2 all  2  251681   98859
feats_plus_cn2 all  5  251681   98859
feats_plus_cn2 all 10  251681   98859
feats_plus_cn2 all 15  251681   98859
feats_plus_cn2 all 20  251681   98859
feats_plus_cn2 day  1  174044   53074
feats_plus_cn2 day  2  174044   53074
feats_plus_cn2 day  5  174044   53074
feats_plus_cn2 day 10  174044   53074
feats_plus_cn2 day 15  174044   53074
feats_plus_cn2 day 20  174044   53074
feats_plus_cn2 ngt  1   77637   45785
feats_plus_cn2 ngt  2   77637   45785
feats_plus_cn2 ngt  5   77637   45785
feats_plus_cn2 ngt 10   77637   45785
feats_plus_cn2 ngt 15   77637   45785
feats_plus_cn2 ngt 20   77637   45785
feats_mnus_cn2 all  1  277105  103478
feats_mnus_cn2 all  2  277105  103478
feats_mnus_cn2 all  5  277105  103478
feats_mnus_cn2 all 10  277105  103478
feats_mnus_cn2 all 15  277105  103478
feats_mnus_cn2 all 20  277105  103478
feats_mnus_cn2 day  1  197939   56874
feats_mnus_cn2 day  2  197939   56874
feats_mnus_c

### train and test subroutine

In [17]:
def train_and_test(train_df, test_df, feats, label):
    regr = RandomForestRegressor(n_estimators=100, random_state=0)
    forest = regr.fit(train_df[feats], train_df[label])
#     r2 = regr.score(test_df[feats], test_df[label])
    preds = regr.predict(test_df[feats])
    r2 = r2_score(test_df[label], preds)
    sq_err = mean_squared_error(test_df[label], preds)
    perc_err = mean_absolute_percentage_error(test_df[label], preds)
    return {'forest': forest, 'preds': preds, 'r2': r2, 'sq_err': sq_err, 'perc_err': perc_err}

### Get All Results

In [ ]:
results = {}
for f in feats_map.keys(): 
    results[f] = {}
    for r in r0_map.keys():
        results[f][r] = {}
        for m in smooth_mins:
            print("Progress: {} {} {}".format(f, r, m))
            label = '{}_{}T'.format(r0_map[r], str(m))
            feats = feats_map[f]
            valid = valid_masks[f][r][m]
            results[f][r][m] = train_and_test(df_subset.loc[train & valid], df_subset.loc[test & valid], feats, label)
            with open('data.merged.smoothed.resampled.pkl', 'wb') as fh:
                pickle.dump(results, fh)

Progress: feats_plus_cn2 all 1


### Save File

In [ ]:
with open('data.merged.smoothed.resampled.pkl', 'wb') as fh:
    pickle.dump(results, fh)

### Compare all Results

In [ ]:
# for r in [ results_all_1T, results_all_2T, results_all_5T, results_all_10T, results_all_15T, results_all_20T ]:
#     print(f"{r['r2']:.5},{r['sq_err']:.5},{r['perc_err']:.5}")

In [ ]:
# for r in [ results_day_1T, results_day_2T, results_day_5T, results_day_10T, results_day_15T, results_day_20T ]:
#     print(f"{r['r2']:.5},{r['sq_err']:.5},{r['perc_err']:.5}")

In [ ]:
# for r in [ results_night_1T, results_night_2T, results_night_5T, results_night_10T, results_night_15T, results_night_20T ]:
#     print(f"{r['r2']:.5},{r['sq_err']:.5},{r['perc_err']:.5}")

In [ ]:
# scatter_with_errors(test_truth_day, test_preds_day, test_perc_err_day, xmin, xmax, ymin, ymax)
# plot_importance(regr, df_subset.loc[train&valid_day,feats], feats)

In [ ]:
# scatter_with_errors(test_truth_day, test_preds_day, test_perc_err_day, xmin, xmax, ymin, ymax)
# plot_importance(regr, df_subset.loc[train&valid_day,feats], feats)

In [ ]:
# error_by_r0_histograms(test_truth_all, error_perc(test_truth_all, test_preds_all), 0, 80)
# error_by_r0_histograms(test_truth_day, error_perc(test_truth_day, test_preds_day), 0, 80)
# error_by_r0_histograms(test_truth_night, error_perc(test_truth_night, test_preds_night), 0, 80)

In [ ]:
# test_truth_night = df_subset.loc[test&valid_night,label_night]
# test_truth_day = df_subset.loc[test&valid_day,label_day]
# test_truth_all = df_subset.loc[test&valid,label]

In [ ]:
# test_truth_night_15T = df_subset.loc[test&valid_night,'r0_night_15T']
# test_pred_night_15T = results_night_10T['preds']
# error_by_r0_histograms(test_truth_night_15T, error_perc(test_truth_night_15T, test_pred_night_15T), 0, 80)
# scatter_with_errors(test_truth_night_15T, test_pred_night_15T, error_perc, xmin, xmax, ymin, ymax)

In [ ]:
# test_truth_day_15T = df_subset.loc[test&valid_day,'r0_day_15T']
# test_pred_day_15T = results_day_15T['preds']
# error_by_r0_histograms(test_truth_day_15T, error_perc(test_truth_day_15T, test_pred_day_15T), 0, 90)
# scatter_with_errors(test_truth_day_15T, test_pred_day_15T, error_perc, xmin, xmax, ymin, ymax)

### Plotting

#### scatter plots of actual vs. predict using error_diff

In [ ]:
# %matplotlib inline
# scatter_with_errors(test_truth_all, test_preds_all, error_perc, xmin, xmax, ymin, ymax)

# scatter_with_errors(test_truth_night, test_preds_night, error_perc, xmin, xmax, ymin, ymax)
# plot_importance(regr, df_subset.loc[train&valid_night,feats], feats)

# scatter_with_errors(test_truth_night, test_preds_night, error_perc, xmin, xmax, ymin, ymax)
# plot_importance(regr, df_subset.loc[train&valid_night,feats], feats)

#### interactive time domain plot of errors

switching matplotlib to notebook mode to enable a zoom-in of different portions of the time axis

In [ ]:
# %matplotlib notebook 
# plot_errors_in_time(test_truth_all, test_preds_all)

#### feature importance

feature importance from the model

In [ ]:
# %matplotlib inline

In [ ]:
# plot_importance(regr, df_subset.loc[train&valid,feats], feats)

## debug why CN2 is so low

#### What happens if I drop month and SZA

Answer: turns out we had to take the log of CN2

In [ ]:
# feats_no_sza = ['pressure', 'relative_humidity', 'temperature', 'wind_speed', 'logCn2']
# preds_all_no_sza, r2_all_no_sza = train_and_test(df_subset.loc[train & valid], df_subset.loc[test & valid], feats_no_sza, label)
# scatter_with_errors(test_truth_all, preds_all_no_sza, error_perc, xmin, xmax, ymin, ymax)
# plot_importance(regr, df_subset.loc[train&valid,feats_no_sza], feats_no_sza)
# r2_all_no_sza

#### correlation between the signals using [stats.pearsonr](https://towardsdatascience.com/four-ways-to-quantify-synchrony-between-time-series-data-b99136c4a9c9)

We calculate:
- overall synchrony between r0 and Cn2
- local synchrony between r0 and Cn2

#### Overall Synchrony

In [ ]:
    # r, p = print_pearsonr(df_subset.loc[train&valid,label], df_subset.loc[train&valid,'logCn2'])
# plot_overall_synchrony(df_subset.loc[train&valid,label], df_subset.loc[train&valid,['logCn2']], label, 'logCn2', r)

#### Local Synchrony

In [ ]:
# plot_local_synchrony(df_subset.loc[train&valid,label], df_subset.loc[train&valid,['logCn2']], label, 'Cn2')

## Synchrony using only R0 daytime data

#### Overall Synchrony

In [ ]:
# r, p = print_pearsonr(df_subset.loc[train&valid_day,label_day], df_subset.loc[train&valid_day,'logCn2'])
# r

In [ ]:
# plot_overall_synchrony(df_subset.loc[train&valid_day,label_day], df_subset.loc[train&valid_day,['logCn2']], label_day, 'logCn2', r)

#### Local Synchrony

In [ ]:
# plot_local_synchrony(df_subset.loc[train&valid_day,label_day], df_subset.loc[train&valid_day,['logCn2']], label_day, 'logCn2')

## Try comparing results without CN2

In [ ]:
# results_all_15T_nocn2 = train_and_test(df_subset.loc[train & valid], df_subset.loc[test & valid], feats_minus_cn2, 'r0_15T')

In [ ]:
# results_day_15T_nocn2 = train_and_test(df_subset.loc[train & valid_day], df_subset.loc[test & valid_day], feats_minus_cn2, 'r0_day_15T')

In [ ]:
# results_night_15T_nocn2 = train_and_test(df_subset.loc[train & valid_night], df_subset.loc[test & valid_night], feats_minus_cn2, 'r0_night_15T')

In [ ]:
# for r in [ results_all_15T, results_all_15T_nocn2 ]:
#     print(f"{r['r2']:.5},{r['sq_err']:.5},{r['perc_err']:.5}")

In [ ]:
# for r in [ results_day_15T, results_day_15T_nocn2 ]:
#     print(f"{r['r2']:.5},{r['sq_err']:.5},{r['perc_err']:.5}")

In [ ]:
# for r in [ results_night_15T, results_night_15T_nocn2 ]:
#     print(f"{r['r2']:.5},{r['sq_err']:.5},{r['perc_err']:.5}")

In [ ]:
# plot_importance(results_all_15T['forest'], df_subset.loc[train & valid, feats], feats)

In [ ]:
# test_truth_night_15T = df_subset.loc[test&valid_night,'r0_night_15T']
# test_pred_night_15T = results_night_10T['preds']
# error_by_r0_histograms(test_truth_night_15T, error_perc(test_truth_night_15T, test_pred_night_15T), 0, 80)
# scatter_with_errors(test_truth_night_15T, test_pred_night_15T, error_perc, xmin, xmax, ymin, ymax)

## Why R2 so weird?

In [ ]:
# print("{:.5}".format(np.var(df_subset.loc[test&valid_night,'r0_night'])))
# print("{:.5}".format(np.var(df_subset.loc[test&valid_night,'r0_night_2T'])))
# print("{:.5}".format(np.var(df_subset.loc[test&valid_night,'r0_night_5T'])))
# print("{:.5}".format(np.var(df_subset.loc[test&valid_night,'r0_night_10T'])))
# print("{:.5}".format(np.var(df_subset.loc[test&valid_night,'r0_night_15T'])))
# print("{:.5}".format(np.var(df_subset.loc[test&valid_night,'r0_night_20T'])))

## Plot One Day

In [ ]:
# def plot_one_day(preds, smooth_df, orig_df, startdate, enddate):

#     preds_df = pd.DataFrame(preds, index=smooth_df.index) 

#     daymask = (orig_df.index > startdate ) & (orig_df.index < enddate)

#     plt.figure(figsize=(20, 5))
#     plt.plot(orig_df[daymask], 'g.', label='r0')
#     plt.plot(smooth_df[daymask], 'b.', label='r0 smoothed (15min)')
#     plt.plot(preds_df[daymask], 'r.', label='preds')
#     plt.ylabel('r0')
#     plt.legend()
#     return

In [ ]:
# plot_one_day(results_all_15T['preds'], df_subset.loc[test & valid, 'r0_15T'], df_subset.loc[test & valid, 'r0'], '2020-05-30 04:00', '2020-05-31')


In [ ]:
# plot_one_day(results_all_15T['preds'], df_subset.loc[test & valid, 'r0_15T'], df_subset.loc[test & valid, 'r0'], '2020-08-30 04:00', '2020-08-31 00:00')

In [ ]:
# plot_one_day(results_all_15T['preds'], df_subset.loc[test & valid, 'r0_15T'], df_subset.loc[test & valid, 'r0'], '2020-02-15 04:00', '2020-02-16 00:00')

In [ ]:
# plot_one_day(results_all_15T['preds'], df_subset.loc[test & valid, 'r0_15T'], df_subset.loc[test & valid, 'r0'], '2020-11-15 04:00', '2020-11-16 00:00')